<a href="https://colab.research.google.com/github/subhashbs36/DeepLearning_Colab/blob/main/Subhash_minst_neural_net1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)


In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

In [ ]:
valid_threes = (path/'valid'/'3').ls().sorted()
valid_sevens = (path/'valid'/'7').ls().sorted()
valid_threes

In [ ]:
stacked_threes = torch.stack([tensor(Image.open(i)) for i in threes]).float()/255
stacked_sevens = torch.stack([tensor(Image.open(i)) for i in sevens]).float()/255

In [ ]:
stacked_valid_threes = torch.stack([tensor(Image.open(i)) for i in valid_threes]).float()/255
stacked_valid_sevens = torch.stack([tensor(Image.open(i)) for i in valid_sevens]).float()/255

In [ ]:
x_train = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
y_train = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
x_train.shape, y_train.shape

In [ ]:
x_valid = torch.cat([stacked_valid_threes, stacked_valid_sevens]).view(-1, 28*28)
y_valid = tensor([1]*len(valid_threes) + [0]*valid_sevens).unsqueeze(1)
x_valid.shape, y_valid.shape

In [ ]:
dset = list(zip(x_train, y_train))
valid_dset = list(zip(x_valid, y_valid))

dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

In [ ]:
linear_model = nn.Linear(28*28, 1)
lr = 0.01

In [ ]:
def mnist_loss(predicted, target):
  predicted = predicted.sigmoid()
  return torch.where(target==1, 1-predicted, predicted).mean()

In [ ]:
class basicOpti():
  def __init__(self, params, lr):
    self.params = params
    self.lr = lr

  def step(self):
    for p in self.params:
      p.data -= p.grad * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad.zero_()

In [ ]:
def calc_grad(xb, yb, model):
  pred = model(xb)
  loss = mnist_loss(pred, yb)
  loss.backward()

In [ ]:
def batch_accuracy(xa, ya):
  predicted = xa.sigmoid()
  correct = (predicted>0.5) == ya
  return correct.float.mean()

In [ ]:
optimizer = basicOpti(linear_model.parameters(), lr)

In [ ]:
def train_epoch(model):
  for xa, ya in dl:
    calc_grad(xa, ya, model)
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
def validate_epoch(model):
  accuracy = [batch_accuracy(model(xa), ya) for xa, ya in valid_dl]
  return round(torch.stack(accuracy).mean().item(), 4)

In [ ]:
def train_model(model, epochs):
  for epoch in epochs:
    train_epoch(model)
    print(validate_epoch(model), end=' ')